## 1. Simple Binary Classification with defaults

In this notebook we will use the Adult Census dataset. Download the data from [here](https://www.kaggle.com/wenruliu/adult-income-dataset/downloads/adult.csv/2).

In [1]:
import numpy as np
import pandas as pd
import torch

from pytorch_widedeep.preprocessing import WidePreprocessor, DeepPreprocessor
from pytorch_widedeep.models import Wide, DeepDense, WideDeep
from pytorch_widedeep.metrics import BinaryAccuracy

In [2]:
df = pd.read_csv('data/adult/adult.csv.zip')
df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [3]:
# For convenience, we'll replace '-' with '_'
df.columns = [c.replace("-", "_") for c in df.columns]
# binary target
df['income_label'] = (df["income"].apply(lambda x: ">50K" in x)).astype(int)
df.drop('income', axis=1, inplace=True)
df.head()

,age,workclass,fnlwgt,education,educational_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_label
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,0
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,0
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,1
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,1
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,0


### 1.1 Preparing the data

Have a look to notebooks one and two if you want to get a good understanding of the next few lines of code (although there is no need to use the package)

In [4]:
wide_cols = ['education', 'relationship','workclass','occupation','native_country','gender']
crossed_cols = [('education', 'occupation'), ('native_country', 'occupation')]
cat_embed_cols = [('education',16), ('relationship',8), ('workclass',16),
    ('occupation',16),('native_country',16)]
continuous_cols = ["age","hours_per_week"]
target_col = 'income_label'

In [5]:
# TARGET
target = df[target_col].values

# WIDE
preprocess_wide = WidePreprocessor(wide_cols=wide_cols, crossed_cols=crossed_cols)
X_wide = preprocess_wide.fit_transform(df)

# DEEP
preprocess_deep = DeepPreprocessor(embed_cols=cat_embed_cols, continuous_cols=continuous_cols)
X_deep = preprocess_deep.fit_transform(df)

In [6]:
print(X_wide)
print(X_wide.shape)

[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(48842, 796)


In [7]:
print(X_deep)
print(X_deep.shape)

[[ 0.          0.          0.         ...  0.         -0.99512893
  -0.03408696]
 [ 1.          1.          0.         ...  0.         -0.04694151
   0.77292975]
 [ 2.          1.          1.         ...  0.         -0.77631645
  -0.03408696]
 ...
 [ 1.          3.          0.         ...  0.          1.41180837
  -0.03408696]
 [ 1.          0.          0.         ...  0.         -1.21394141
  -1.64812038]
 [ 1.          4.          6.         ...  0.          0.97418341
  -0.03408696]]
(48842, 7)


### 1.2. Defining the model

In [8]:
wide = Wide(wide_dim=X_wide.shape[1], output_dim=1)
deepdense = DeepDense(hidden_layers=[64,32], 
                      deep_column_idx=preprocess_deep.deep_column_idx,
                      embed_input=preprocess_deep.embeddings_input,
                      continuous_cols=continuous_cols)
model = WideDeep(wide=wide, deepdense=deepdense)

In [9]:
model

WideDeep(
  (wide): Wide(
    (wide_linear): Linear(in_features=796, out_features=1, bias=True)
  )
  (deepdense): Sequential(
    (0): DeepDense(
      (embed_layers): ModuleDict(
        (emb_layer_education): Embedding(16, 16)
        (emb_layer_native_country): Embedding(42, 16)
        (emb_layer_occupation): Embedding(15, 16)
        (emb_layer_relationship): Embedding(6, 8)
        (emb_layer_workclass): Embedding(9, 16)
      )
      (embed_dropout): Dropout(p=0.0, inplace=False)
      (dense): Sequential(
        (dense_layer_0): Sequential(
          (0): Linear(in_features=74, out_features=64, bias=True)
          (1): LeakyReLU(negative_slope=0.01, inplace=True)
          (2): Dropout(p=0.0, inplace=False)
        )
        (dense_layer_1): Sequential(
          (0): Linear(in_features=64, out_features=32, bias=True)
          (1): LeakyReLU(negative_slope=0.01, inplace=True)
          (2): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (1): Linear(in_features=32

As you can see, the model is not particularly complex. In mathematical terms (Eq 3 in the [original paper](https://arxiv.org/pdf/1606.07792.pdf)): 

$$
pred = \sigma(W^{T}_{wide}[x, \phi(x)] + W^{T}_{deep}a_{deep}^{(l_f)} +  b) 
$$ 


The architecture above will output the 1st and the second term in the parenthesis. `WideDeep` will then add them and apply an activation function (`sigmoid` in this case). For more details, please refer to the paper.

### 1.3 Compiling and Running/Fitting
Once the model is built, we just need to compile it and run it

In [10]:
model.compile(method='binary', metrics=[BinaryAccuracy])

In [11]:
model.fit(X_wide=X_wide, X_deep=X_deep, target=target, n_epochs=5, batch_size=256, val_split=0.2)

  0%|          | 0/153 [00:00<?, ?it/s]

Training


valid: 100%|██████████| 39/39 [00:00<00:00, 115.33it/s, loss=0.348, metrics={'acc': 0.8406}]


As you can see, you can run a wide and deep model in just a few lines of code

Let's now see how to use `WideDeep` with varying parameters

## 2. Binary Classification with varying parameters

### 2.1 Warm-up

We can choose to warm up each model individually before the joined training begins. To warm up, the models will be trained during `warm_epochs` using a triangular one-cycle learning rate (referred as *slanted triangular learning rates* in [Howard & Ruder 2018](https://arxiv.org/pdf/1801.06146.pdf)) going from `warm_max_lr`/10. to `warm_max_lr` (default is 0.01). 10% of the training steps are used to increase the learning rate which then decreases back to `warm_max_lr`/10. for the remaining 90%.   

In [12]:
wide = Wide(wide_dim=X_wide.shape[1], output_dim=1)
deepdense = DeepDense(hidden_layers=[64,32], 
                      deep_column_idx=preprocess_deep.deep_column_idx,
                      embed_input=preprocess_deep.embeddings_input,
                      continuous_cols=continuous_cols)
model = WideDeep(wide=wide, deepdense=deepdense)
model.compile(method='binary', metrics=[BinaryAccuracy])
model.fit(X_wide=X_wide, X_deep=X_deep, target=target, n_epochs=4, batch_size=256, val_split=0.2, 
          warm_up=True, warm_epochs=1)

  0%|          | 0/153 [00:00<?, ?it/s]

Warming up wide for 1 epochs


  0%|          | 0/153 [00:00<?, ?it/s]

Warming up deepdense for 1 epochs


  0%|          | 0/153 [00:00<?, ?it/s]

Training


valid: 100%|██████████| 39/39 [00:00<00:00, 111.94it/s, loss=0.347, metrics={'acc': 0.843}]


###  2.1 Dropout and Batchnorm

In [13]:
wide = Wide(wide_dim=X_wide.shape[1], output_dim=1)
# We can add dropout and batchnorm to the dense layers
deepdense = DeepDense(hidden_layers=[64,32], dropout=[0.5, 0.5], batchnorm=True,
                      deep_column_idx=preprocess_deep.deep_column_idx,
                      embed_input=preprocess_deep.embeddings_input,
                      continuous_cols=continuous_cols)
model = WideDeep(wide=wide, deepdense=deepdense)

In [14]:
model

WideDeep(
  (wide): Wide(
    (wide_linear): Linear(in_features=796, out_features=1, bias=True)
  )
  (deepdense): Sequential(
    (0): DeepDense(
      (embed_layers): ModuleDict(
        (emb_layer_education): Embedding(16, 16)
        (emb_layer_native_country): Embedding(42, 16)
        (emb_layer_occupation): Embedding(15, 16)
        (emb_layer_relationship): Embedding(6, 8)
        (emb_layer_workclass): Embedding(9, 16)
      )
      (embed_dropout): Dropout(p=0.0, inplace=False)
      (dense): Sequential(
        (dense_layer_0): Sequential(
          (0): Linear(in_features=74, out_features=64, bias=True)
          (1): LeakyReLU(negative_slope=0.01, inplace=True)
          (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (3): Dropout(p=0.5, inplace=False)
        )
        (dense_layer_1): Sequential(
          (0): Linear(in_features=64, out_features=32, bias=True)
          (1): LeakyReLU(negative_slope=0.01, inplace=True)
    

We can use different initializers, optimizers and learning rate schedulers for each `branch` of the model

###  2.1 Optimizers, LR schedulers, Initializers and Callbacks

In [15]:
from pytorch_widedeep.initializers import KaimingNormal, XavierNormal
from pytorch_widedeep.callbacks import ModelCheckpoint, LRHistory, EarlyStopping
from pytorch_widedeep.optim import RAdam

In [16]:
# Optimizers
wide_opt = torch.optim.Adam(model.wide.parameters())
deep_opt = RAdam(model.deepdense.parameters())
# LR Schedulers
wide_sch = torch.optim.lr_scheduler.StepLR(wide_opt, step_size=3)
deep_sch = torch.optim.lr_scheduler.StepLR(deep_opt, step_size=5)

the component-dependent settings must be passed as dictionaries, while general settings are simply lists

In [17]:
# Component-dependent settings as Dict
optimizers = {'wide': wide_opt, 'deepdense':deep_opt}
schedulers = {'wide': wide_sch, 'deepdense':deep_sch}
initializers = {'wide': KaimingNormal, 'deepdense':XavierNormal}
# General settings as List
callbacks = [LRHistory(n_epochs=10), EarlyStopping, ModelCheckpoint(filepath='model_weights/wd_out')]
metrics = [BinaryAccuracy]

In [18]:
model.compile(method='binary', optimizers=optimizers, lr_schedulers=schedulers, 
              initializers=initializers,
              callbacks=callbacks,
              metrics=metrics)

In [19]:
model.fit(X_wide=X_wide, X_deep=X_deep, target=target, n_epochs=10, batch_size=256, val_split=0.2)

  0%|          | 0/153 [00:00<?, ?it/s]

Training


valid: 100%|██████████| 39/39 [00:00<00:00, 109.54it/s, loss=0.356, metrics={'acc': 0.8212}]


In [20]:
dir(model)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_activation_fn',
 '_apply',
 '_backward_hooks',
 '_buffers',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_get_name',
 '_load_from_state_dict',
 '_load_state_dict_pre_hooks',
 '_loss_fn',
 '_lr_scheduler_step',
 '_modules',
 '_named_members',
 '_parameters',
 '_predict',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_save_to_state_dict',
 '_slow_forward',
 '_state_dict_hooks',
 '_tracing_name',
 '_train_val_split',
 '_training_step',
 '_validation_step',
 '_version',
 '_warm_model',
 '_warm_up',
 'add_module',
 'apply',
 'batch_size',
 'buffers',
 'cal

You see that, among many methods and attributes we have the `history` and `lr_history` attributes

In [21]:
model.history.epoch

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [22]:
print(model.history._history)

{'train_loss': [0.9105595845023012, 0.5782874746649873, 0.47749636551133945, 0.4281357573527916, 0.4061133719347661, 0.3947428677206725, 0.3914796267849168, 0.38961343983419583, 0.38926642879941104, 0.386191635934356], 'train_acc': [0.5729, 0.7309, 0.7783, 0.8012, 0.8087, 0.8126, 0.813, 0.8169, 0.8162, 0.8171], 'val_loss': [0.5205524701338547, 0.4024948156796969, 0.37283383500881684, 0.36488463634099716, 0.35886253301913923, 0.35753893775817674, 0.35730381042529374, 0.35672229528427124, 0.3563888867696126, 0.35624249776204425], 'val_acc': [0.6051, 0.7466, 0.7879, 0.807, 0.814, 0.8171, 0.8175, 0.8207, 0.8203, 0.8212]}


In [23]:
print(model.lr_history)

{'lr_wide_0': [0.001, 0.001, 0.001, 0.0001, 0.0001, 0.0001, 1.0000000000000003e-05, 1.0000000000000003e-05, 1.0000000000000003e-05, 1.0000000000000002e-06], 'lr_deepdense_0': [0.001, 0.001, 0.001, 0.001, 0.001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001]}


We can see that the learning rate effectively decreases by a factor of 0.1 (the default) after the corresponding `step_size`. Note that the keys of the dictionary have a suffix `_0`. This is because if you pass different parameter groups to the torch optimizers, these will also be recorded. We'll see this in the `Regression` notebook. 

And I guess one has a good idea of how to use the package. Before we leave this notebook just mentioning that the `WideDeep` class comes with a useful method to "rescue" the learned embeddings. For example, let's say I want to use the embeddings learned for the different levels of the categorical feature `education`

In [24]:
model.get_embeddings(col_name='education', cat_encoding_dict=preprocess_deep.encoding_dict)

{'11th': array([ 0.41717738,  0.33800027,  0.14558531,  0.23621577,  0.18866219,
         0.41959327, -0.40843502,  0.2764823 , -0.04228376, -0.2770995 ,
        -0.04413053,  0.21518119,  0.54168355,  0.06607324, -0.25075328,
         0.43814617], dtype=float32),
 'HS-grad': array([-0.10619222,  0.08463281, -0.1720976 ,  0.16436636, -0.15246172,
        -0.15992908, -0.07142664,  0.02854085, -0.38450843,  0.3621033 ,
         0.00361137, -0.37991726, -0.00742414, -0.19315098,  0.23940389,
         0.00427438], dtype=float32),
 'Assoc-acdm': array([-0.10346556,  0.5880965 , -0.35604322, -0.28074315,  0.11279969,
        -0.03097979,  0.1316176 ,  0.04005286,  0.22053859,  0.2822993 ,
         0.2548561 , -0.00729926,  0.16980447,  0.00099144, -0.21386623,
        -0.03788675], dtype=float32),
 'Some-college': array([ 1.0774918e-01, -5.1934827e-02,  2.2199769e-01,  1.2707384e-01,
        -6.6714182e-02, -1.2450726e-02, -4.7941156e-02, -1.1758558e-02,
        -1.7372087e-02, -3.7972507e-